<a href="https://colab.research.google.com/github/kanishk16/routetoMIDASS/blob/master/Python_Problem/Python_Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

>**Necessary Installations & Imports**

In [2]:
!pip install jsonlines

In [0]:
import tweepy
import jsonlines

In [0]:
!wget -c https://github.com/wbolster/jsonlines/tree/1.2.0

--2019-04-06 18:58:10--  https://github.com/wbolster/jsonlines/tree/1.2.0
Resolving github.com (github.com)... 192.30.255.112, 192.30.255.113
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘1.2.0’

1.2.0                   [ <=>                ]  82.30K  --.-KB/s    in 0.01s   

2019-04-06 18:58:10 (5.48 MB/s) - ‘1.2.0’ saved [84279]



> **Task 1a : A python script to fetch all the tweets(as many as allowed by Twitter API) done by @midasIIITD twitter handle and dump the responses into JSONlines file**

> **Authorization for Twitter API**

In [0]:
"""
Stores tokens and keys
"""
consumer_key = "f69Ch2S4kVRsXbaZwhs0Qhc7M"
consumer_secret = "Z43Y85EwvKe3tOHVNAfvxEx7r7jmNliHhXDgZsdSnXuRIZKLKz"
access_key = "1034447027923374082-ftqEyERgMoJ8s8sMFICXfxy7UC44vs"
access_secret = "p8hg6CRuXVzn9yFxrnwObSCrm0gmW2PhKiveqw9SLwzsg"


> **Function to fetch all tweets**

In [5]:
# Function to extract tweets 
def get_tweets(username): 
          
        # Authorization to consumer key and consumer secret 
        auth = tweepy.OAuthHandler(consumer_key, consumer_secret) 
  
        # Access to user's access key and access secret 
        auth.set_access_token(access_key, access_secret) 
  
        # Calling api 
        api = tweepy.API(auth) 
  
        # tweets to be extracted 
        number_of_tweets=5
        tweets = api.user_timeline(screen_name=username) 
  
        # Empty Array 
        tmp=[]  
        
        images = 0
        for tweet in tweets:
          tweet_media = str(tweet.entities) 
          images = tweet_media.count("photo")
          #if "photo" in tweet_media:
          #  images += 1 
          print(images)
  
        # create array of tweet information: text, date/time, favorites/likes, retweets, media 
        tweets_for_csv = [[tweet.text, str(tweet.created_at), tweet.favorite_count, tweet.retweet_count, images] for tweet in tweets] 
        for j in tweets_for_csv: 
            
            #print(type(tweets_for_csv))
            # Appending tweets to the empty array tmp 
            tmp.append(j)  
           
        # Open a JSON file to dump all the tweets
        with jsonlines.open('/content/twitter1.jsonl', mode='w') as writer:
          writer.write_all(tmp)
        
        # Printing the tweets 
        print("RECORDED") 
  
  
if __name__ == '__main__': 
  
    # Twitter handle whose tweets are to be extracted 
    get_tweets("midasIIITD") 

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
RECORDED


In [7]:
!pip install pandas

In [9]:
import pandas as pd

with jsonlines.open('/content/twitter1.jsonl', mode='r') as reader:
  #print(reader.read())
  #print(type(reader))
  labels = ['text','time','likes','retweets', 'images']
  df = pd.DataFrame(columns =labels)
  for r in reader:
    print(r)
    #df1 = df.append(pd.Series([r], index = labels), ignore_index=True)
    #print(df1)
    df = pd.DataFrame([r], columns = labels).append(df, ignore_index=True)
    #df1= df.append(pd.Series([r], index=labels)
    #print(df1.tolist(), columns =labels)
print(df)  

['RT @IIITDelhi: We are delighted to share that IIIT-Delhi is ranked 55 by NIRF this year. We have moved up by 11 positions compared to the p…', '2019-04-09 16:45:07', 0, 9, 0]
['RT @Harvard: Professor Jelani Nelson founded AddisCoder, a program that teaches students in Ethiopia how to code https://t.co/0sM06p4qxw', '2019-04-09 05:04:27', 0, 35, 0]
['RT @emnlp2019: For anyone interested in submitting to EMNLP 2019, the anonymity period begins in less two weeks, on April 21.\nhttps://t.co/…', '2019-04-09 05:04:11', 0, 12, 0]
['RT @multimediaeval: Announcing the 2019 MediaEval multimedia tasks: https://t.co/V6NMt1vaNV Participation is open for anyone who is interes…', '2019-04-08 19:38:09', 0, 15, 0]
['Many Congratulations to @midasIIITD student, Shagun Uppal @shagunuppls, on getting selected for the summer internsh… https://t.co/bzhiSm4zuB', '2019-04-08 07:08:12', 14, 2, 0]
['@midasIIITD thanks all students who have appeared for the interview yesterday. We will announce the interview re